# **Optiver Realized Volatility Prediction**&#x1f600;
データサイエンスのスキルを応用して、金融市場を改善します


## ※　English version is here.

https://www.kaggle.com/chumajin/optiver-realized-eda-for-starter-english-version


## このコンペは、株のボラティリティ(価格変動の度合い)を時間(time id)ごとに予測するコンペだと思います。


## 少しでもお役に立てば、**upvote**して頂けると嬉しいです。
　※　以前も私のnotebookにupvoteしてくれた方ありがとうございます。
 
 
 
 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 何を予測するか(submission.csvから見ます)

In [ ]:
sample = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")
sample

### row_idとtargetしかありません。
### targetはボラティリティ(価格変動の度合い)でそれを予測するコンペなのだと思います。
### row_idとは何かは次を参照。

# 2. 何から予測するか(test.csvから見ます)

In [ ]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

#### submissionファイルのrow_idはstock_id(株のid)とtime_id(時間のid)を-でつなげたものということがわかる。


#### さらにtest.csvには、book_test.parquetとtrade_test.parquetというものがついています。今回はstock_id = 0のものだけ。中身を見てみる
※　ファイルパスでも開くけど、その上のフォルダまでのpathで開くみたいです。

In [ ]:
book_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_testparquet

In [ ]:
trade_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_testparquet

#### これらの情報から、株のボラティリティ(価格変動の度合い)を当てるコンペだと思います。
#### 以上を踏まえて、各項目の解説を含めてtrain.csvを見ていき,EDAしてお試しでsubmitしていきます。

# 3. train.csv

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

#### とても単純な構成。株のstock_idとtime_id、targetの値が出ています。
#### targetはDataに解説あるように、10分間のtotalボラティリティだと思います（これを学習のtargetとします。）

# 4. train.parquet

#### testデータでの解説と同様に、train.csvに付随したファイルとして、book_train.parquetとtrade_train.parquetいうファイルがついています。
#### 例えばtrain.csvの一番上の株の銘柄 stock_id=0のそれぞれのparquetファイルを見てみます。

## **4.1 book_train.parquet**


#### 市場に投入された最も競争力のある売買注文に関するオーダーブックデータを提供します。
#### bookの上位2つのレベルが共有されます。bookの第1レベルは価格面でより競争力があり、第2レベルよりも実行が優先されます。


******補足(個人的解釈)******

オーダーbookデータは予約状況だと思います。この値段まで下がってきたら、買う。上がってきたら売るみたいな予約情報です。

In [ ]:
book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
book_example

Bid（売値） は、買い手が株を買う上で希望する価格で、Ask（買値）は、売り手が株を売る上で希望する価格です。
わかりやすく言い換えると、

* stock_id : 株の銘柄（どの株か）
* time_id : どの時間の情報かのid (submissionファイルのtime_idと連動しています)
* seconds_in_bucket : time_idの中で、0からスタートして何秒後か。たぶん予測するのは、10分のtotalなので、seconds_in_bucketは、最大600 secのはず
* bid_price1,2 : 株の買値の希望値の１番目と２番目
※　(Normalized prices of the most/second most competitive buy level. だから、正確には、１番と２番目に正規化されたレベルの買値。

→買値の希望値をみんな出しているけど、それの正規化したときに一番多い値と２番目に多い値と推測。以下askも逆の現象。)


* ask_price1,2 : 株の売り値の希望値
* bid_size1,2 : 買うのを希望している側の１番目と２番目の株式数
* ask_size1,2 : 売るのを希望している側の１番目と２番目の株式数


※　parquetはロード時にカテゴリデータ型に強制することに注意が必要らしいです。


## 4.2 **trade_train.parquet**

#### 実際に実行された取引に関するデータが含まれています。
#### 通常、市場では、実際の取引よりも受動的な売買意図の更新（bookの更新）が多いため、このファイルは注文書よりもまばらであると予想される場合があります。

******補足(個人的解釈)******

リアルタイムで実際に取引された量。買い手がこの値段でこの数量買い、売り手がこの値段でこの数量売ったと推測。

In [ ]:
trade_example = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_example

* stock_id - 同上。
* time_id - 同上。
* seconds_in_bucket- 同上。トレードデータとブックデータは同じ時間枠から取得され、トレードデータは一般にまばらであるため、このフィールドは必ずしも0から始まるとは限らないことに注意してください。
* price-1秒間に発生する実行済みトランザクションの平均価格。価格は正規化されており、平均は各トランザクションで取引された株式数によって重み付けされています。
* size -取引された株式の総数。
* order_count -発生している固有の取引注文の数。

#### 一番上に示したtestデータもtrainデータと同じ構造です。testデータには、targetだけないので、それを予測します。

# 5. EDA

## 5.1 株式ごとの解析
### 5.1.1 株の数

In [ ]:
train

In [ ]:
for col in train.columns:
    print(col,":",len(train[col].unique()))

## stock_id(株の銘柄) は112種類、time_idは3830種類、targetは414287種類ある

## 5.1.2 株ごとの統計量

In [ ]:
stock = train.groupby("stock_id")["target"].agg(["mean","median","std","count","sum"]).reset_index()
stock

#### mean値とsumだけヒストグラム見てみます。

In [ ]:
print("mean value=" ,stock["mean"].mean())
plt.hist(stock["mean"])

## 平均値は0.003とほぼ0に近い

In [ ]:
print("sum value=" ,stock["sum"].mean())
plt.hist(stock["sum"])

## この期間のtotalボラティリティは、平均14.8, maxだと30を超える銘柄も存在。

--------------以下、stock id = 0のtime_id = 5の10分間のふるまいを見ていきます -------------

## 5.2 time_id内でのBid,Ask,実際の取引のふるまい価格の関係

In [ ]:
book_example

In [ ]:
book_test = book_example[book_example["time_id"]==5]
book_test

## 5.2.1 価格変動 (個別 + 全体を重ねたもの)

#### まずは、オーダーブック情報

In [ ]:
samples = ["bid_price1","bid_price2","ask_price1","ask_price2"]
sizes = ["bid_size1","ask_size1","bid_size2","ask_size2"]

for num,a in enumerate(samples):
    plt.figure(figsize = (20,5))
    
    plt.subplot(8,1,2*num+1)
    
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
    #plt.title(a)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
    
    plt.subplot(8,1,2*(num)+2)
    
    plt.bar(book_test["seconds_in_bucket"],book_test[sizes[num]],color="green",label=sizes[num])
    #plt.title(label=sizes[num])
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)

    
    
    
plt.show()
plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
    
plt.title("all bid/ask price")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)


#### これに実際の取引情報を追記します。

In [ ]:
trade_example

In [ ]:
trade_test = trade_example[trade_example["time_id"]==5]
trade_test.head(5)

#### 全体のグラフに、実際の取引を加えてみる

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,1,1)

for num,a in enumerate(samples):
    
   
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
    
plt.plot(trade_test["seconds_in_bucket"],trade_test["price"],label="trade_parquet",lw=10)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
plt.xlim(0,600)

plt.subplot(2,1,2)
plt.bar(trade_test["seconds_in_bucket"],trade_test["size"],label="trade_parquet_size",color ="green")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
plt.xlim(0,600)


## 紫が実際の取引。オーダーブックのbidとaskの間をうろうろしている。

#### ※bitやaskのところ近づいたら、攻防があって、超えたらブレイクしてなどその辺は専門家に任せます。

#### time id5の10分間の中でも、bidとaskの値は変動していることがわかる。これのばらつきが大きいとボラティリティと関係するかもしれない。
#### 例えば、急に株価を変動するニュースが入った時に、ボラティリティがボーンと上がって、
#### 乱高下するとばらつきとかが増えるのかもと推測。Max-Minとかも重要かも。(ボラティリティを予測するから逆の発想ですね)

## 5.2.2 ボラティリティのイメージ

stock_id = 0で最もボラティリティが低かった時をグラフにしてみます。

In [ ]:
stock0 = train[train["stock_id"]==0]
min_index = stock0["target"].idxmin()
min_time_id = stock0.iloc[min_index]["time_id"]
print("min index is",min_time_id,"min target is",stock0.iloc[min_index]["target"])

In [ ]:
book_test_min = book_example[book_example["time_id"]==min_time_id]
trade_test_min = trade_example[trade_example["time_id"]==min_time_id]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test_min["seconds_in_bucket"],book_test_min[a],label=a)
    
plt.plot(trade_test_min["seconds_in_bucket"],trade_test_min["price"],label="trade_parquet",lw=10)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)

stock_id = 0で最もボラティリティが高かった時をグラフにしてみます。

In [ ]:
stock0 = train[train["stock_id"]==0]
max_index = stock0["target"].idxmax()
max_time_id = stock0.iloc[max_index]["time_id"]
print("max index is",max_time_id,"max target is",stock0.iloc[max_index]["target"])

In [ ]:
book_test_max = book_example[book_example["time_id"]==max_time_id]
trade_test_max = trade_example[trade_example["time_id"]==max_time_id]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test_max["seconds_in_bucket"],book_test_max[a],label=a)
    
plt.plot(trade_test_max["seconds_in_bucket"],trade_test_max["price"],label="trade_parquet",lw=10)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)

#### 縦軸のスケールが全然違うので、実際の取引を重ねてみます。

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(trade_test_min["seconds_in_bucket"],trade_test_min["price"],lw=10,label="min_vol_time")
plt.plot(trade_test_max["seconds_in_bucket"],trade_test_max["price"],lw=10,label = "max_vol_time")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)

## ボラティリティが高いときはこの場合は、10分間でかなりの価格変動を起こしていることがわかる。
#### (ばらつきや、Max-Minはやっぱり大事そう)

# 6.submit

## お試しで株ごとのmedian値を全部入れてsubmitします

## 6.1 株ごとのmedian値のdictionary作成

In [ ]:
stock

In [ ]:
stock2 = stock[["stock_id","median"]]
stock2 = stock2.set_index("stock_id")
stock2

In [ ]:
stock_dict = stock2.to_dict()
# example stock id = 0のmedian値
stock_dict["median"][0]

## 6.2 submitファイルの置き換えとsubmission fileの生成

In [ ]:
sample # sample_submission.csvです

row_idからstock_id抽出

In [ ]:
sample["stock_id"] = [s.split("-")[0] for s in sample["row_id"]]
sample

辞書型の中身を代入

In [ ]:
sample["target"] = [stock_dict["median"][int(s)] for s in sample["stock_id"]]
sample

stock_id削除

In [ ]:
sample = sample.drop("stock_id",axis=1)
sample

In [ ]:
sample.to_csv("submission.csv",index=False)

# 最後まで見ていただいてありがとうございます。※以前も私のnotebookにupvoteしてくれた方ありがとうございます。
# お役に立てば、**upvote**いただけたら嬉しいです！